In [97]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
import urllib.request
import bs4 as bs
import re
import string

In [68]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Messi')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [69]:
# Demos un vistazo
article_text

'\n\nargentine professional footballer\neponyms and public art\nmedia\nfamily\nlionel andrés messi[note 1] (spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 june 1987), also known as leo messi, is an argentine professional footballer who plays as a forward for and captains both major league soccer club inter miami and the argentina national team. widely regarded as one of the greatest players of all time, messi has won a record eight ballon d\'or awards[note 2] and a record six european golden shoes, and in 2020 he was named to the ballon d\'or dream team. until leaving the club in 2021, he had spent his entire professional career with barcelona, where he won a club-record 34 trophies, including ten la liga titles, seven copa del rey titles and the uefa champions league four times.[note 3] with his country, he won the 2021 copa américa and the 2022 fifa world cup. a prolific goalscorer and creative playmaker, messi holds the records for most goals in la liga (474), most hat-tr

In [70]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 137469


In [71]:
# Repaso de regex:
# https://docs.python.org/3/library/re.html

# Para practicar regex:
# https://regex101.com/

# el inicio con 'r' antes de cada string indica que se interprete como raw string
# '\n' es interpretado por Python como salto de linea
# r'\n' es interpretado por Python como el string formado por dos caracteres:
#  backslash y n

# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', article_text) # substituir los números entre corchetes
# (notar que los corchetes son interpretados literalmente por los backlsash)
text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

# probar en regex101 con los patrones anteriores:
# 'Hola [1], [], [ estoy bien   [123]. [12sss]. OK!

In [72]:
# Demos un vistazo
text

' argentine professional footballer eponyms and public art media family lionel andrés messi[note 1] (spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 june 1987), also known as leo messi, is an argentine professional footballer who plays as a forward for and captains both major league soccer club inter miami and the argentina national team. widely regarded as one of the greatest players of all time, messi has won a record eight ballon d\'or awards[note 2] and a record six european golden shoes, and in 2020 he was named to the ballon d\'or dream team. until leaving the club in 2021, he had spent his entire professional career with barcelona, where he won a club-record 34 trophies, including ten la liga titles, seven copa del rey titles and the uefa champions league four times.[note 3] with his country, he won the 2021 copa américa and the 2022 fifa world cup. a prolific goalscorer and creative playmaker, messi holds the records for most goals in la liga (474), most hat-tricks in

**1 - Preprocesamiento**

In [100]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def nltk_process(text):
    # Tokenization
    nltk_tokenList = word_tokenize(text)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = []
    for word in nltk_tokenList:
        nltk_lemmaList.append(lemmatizer.lemmatize(word))

    # Stop words
    nltk_stop_words = set(stopwords.words("english"))
    filtered_sentence = [w for w in nltk_lemmaList if w not in nltk_stop_words]

    # Filter Punctuation
    filtered_sentence = [w for w in filtered_sentence if w not in string.punctuation]
    return filtered_sentence


def separar_oraciones_y_palabras(texto):

    oraciones = nltk.sent_tokenize(texto)

    palabras_por_oracion = [nltk_process(oracion) for oracion in oraciones]

    return palabras_por_oracion

sentence_tokens = separar_oraciones_y_palabras(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
sentence_tokens[:2]

[['argentine',
  'professional',
  'footballer',
  'eponym',
  'public',
  'art',
  'medium',
  'family',
  'lionel',
  'andrés',
  'messi',
  'note',
  '1',
  'spanish',
  'pronunciation',
  'ljoˈnel',
  'anˈdɾes',
  'ˈmesi',
  'ⓘ',
  'born',
  '24',
  'june',
  '1987',
  'also',
  'known',
  'leo',
  'messi',
  'argentine',
  'professional',
  'footballer',
  'play',
  'forward',
  'captain',
  'major',
  'league',
  'soccer',
  'club',
  'inter',
  'miami',
  'argentina',
  'national',
  'team'],
 ['widely',
  'regarded',
  'one',
  'greatest',
  'player',
  'time',
  'messi',
  'ha',
  'record',
  'eight',
  'ballon',
  "d'or",
  'award',
  'note',
  '2',
  'record',
  'six',
  'european',
  'golden',
  'shoe',
  '2020',
  'wa',
  'named',
  'ballon',
  "d'or",
  'dream',
  'team']]

**2 - Crear los vectores (word2vec)**

In [102]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [103]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [104]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [105]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 732


In [106]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 568


**3 - Entrenar embeddings**

In [107]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 178089.0
Loss after epoch 1: 83633.9375
Loss after epoch 2: 82537.0625
Loss after epoch 3: 81629.84375
Loss after epoch 4: 81933.75
Loss after epoch 5: 83311.09375
Loss after epoch 6: 84709.1875
Loss after epoch 7: 85219.375
Loss after epoch 8: 84801.875
Loss after epoch 9: 84315.625
Loss after epoch 10: 83631.4375
Loss after epoch 11: 76192.8125
Loss after epoch 12: 70700.625
Loss after epoch 13: 69797.125
Loss after epoch 14: 69245.375
Loss after epoch 15: 68771.125
Loss after epoch 16: 68238.375
Loss after epoch 17: 68929.875
Loss after epoch 18: 68970.625
Loss after epoch 19: 69032.625


(155638, 288360)

**4 - Ensayar**

In [109]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["champion"], topn=10)

[('uefa', 0.966996431350708),
 ('league', 0.9614094495773315),
 ('16', 0.9523664116859436),
 ('round', 0.9458962082862854),
 ('semi-finals', 0.9395339488983154),
 ('last', 0.9338013529777527),
 ('leg', 0.9275516271591187),
 ('eventual', 0.9275002479553223),
 ('group', 0.9176849722862244),
 ('game', 0.9111947417259216)]

Aqui lo que mas relaciona a champion es la "Champion League" y lo relacionando con ella como las rondas, uefa, etc.

In [113]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["argentina"], topn=10)

[('cristiano', -0.547670841217041),
 ('ronaldo', -0.5523843765258789),
 ('top', -0.5591982007026672),
 ('scorer', -0.5600374341011047),
 ('ballon', -0.5886763334274292),
 ("d'or", -0.5994566082954407),
 ('million', -0.6040346026420593),
 ('liga', -0.633312463760376),
 ('all-time', -0.6333502531051636),
 ('la', -0.6415724754333496)]

Contrariamente a la palabra palabra argentina en primer lugar aparece cristiano ronaldo, igual que ballon d'or, liga

In [123]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["argentina"], topn=10)

[('led', 0.9875391721725464),
 ('germany', 0.9843007922172546),
 ('qualification', 0.9764503240585327),
 ('mexico', 0.9705032706260681),
 ('finish', 0.9698095917701721),
 ('securing', 0.9696104526519775),
 ('paraguay', 0.9682499766349792),
 ('uruguay', 0.9667478203773499),
 ('lead', 0.9646218419075012),
 ('sergio', 0.964072585105896)]

Principalmente se relaciona con lo que se ha enfrentado con argentina

In [122]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["madrid"], topn=20)

[('away', 0.8744895458221436),
 ('atlético', 0.8686951398849487),
 ('lost', 0.8628809452056885),
 ('rival', 0.8557963967323303),
 ('home', 0.8447725772857666),
 ('win', 0.844729483127594),
 ('1–0', 0.8388054966926575),
 ('de', 0.8319650888442993),
 ('real', 0.8308478593826294),
 ('3–0', 0.8280789256095886),
 ('eliminated', 0.8271188735961914),
 ('semi-finals', 0.82567298412323),
 ('5–0', 0.8224913477897644),
 ('leg', 0.8216904401779175),
 ('4–2', 0.8196411728858948),
 ('eventual', 0.8176290392875671),
 ('sevilla', 0.8159175515174866),
 ('last', 0.813927173614502),
 ('defeated', 0.8114623427391052),
 ('1–1', 0.8084837794303894)]

In [124]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["riquelme"])

KeyError: ignored

In [125]:
# el método `get_vector` permite obtener los vectores:
vector_messi = w2v_model.wv.get_vector("messi")
print(vector_messi)

[ 5.82254231e-02  2.44593129e-01 -9.76083160e-04  1.50953025e-01
 -1.12092510e-01 -1.03059053e-01  7.28710219e-02  1.53088331e-01
  4.07482460e-02 -1.59799844e-01 -6.32371157e-02 -1.63199589e-01
  1.87195595e-02  2.44518220e-02 -6.42421171e-02 -1.11242570e-01
 -5.41860759e-02  1.29579306e-01  8.37979838e-02 -5.20474240e-02
 -4.30771103e-03  3.03928163e-02  2.60654464e-02  9.55979452e-02
  1.30944595e-01  1.32201523e-01 -1.38222679e-01  3.69305834e-02
 -1.06408417e-01 -9.80556458e-02  9.33940634e-02 -8.94953907e-02
  1.19280897e-01  4.12440188e-02  1.60169043e-02 -7.40698650e-02
  3.02563328e-02 -5.70817329e-02  6.21580295e-02  6.43202588e-02
  3.05899456e-02  4.68094870e-02 -1.01318710e-01 -1.18545212e-01
 -3.48909162e-02 -3.73619348e-02  6.38611764e-02  7.46666417e-02
  3.95165645e-02  4.38983105e-02 -2.34785937e-02 -7.80609297e-03
 -2.29676604e-01  2.20090240e-01  3.18736490e-03  1.00051962e-01
  7.84241259e-02 -1.64648741e-02 -1.50949508e-01 -3.02254185e-02
 -1.98142566e-02 -6.70771

In [126]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_messi)

[('messi', 1.0),
 ('announced', 0.9770557880401611),
 ('decision', 0.971973717212677),
 ('10', 0.9714443683624268),
 ('confirmed', 0.9696908593177795),
 ('left', 0.9691008925437927),
 ('family', 0.9679230451583862),
 ('birthday', 0.9677523374557495),
 ('early', 0.9670479893684387),
 ('reported', 0.9668148756027222)]

In [127]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["messi"], topn=10)

[('announced', 0.9770557880401611),
 ('decision', 0.971973717212677),
 ('10', 0.9714443683624268),
 ('confirmed', 0.9696908593177795),
 ('left', 0.9691008925437927),
 ('family', 0.9679230451583862),
 ('birthday', 0.9677523374557495),
 ('early', 0.9670479893684387),
 ('reported', 0.9668148756027222),
 ('rosario', 0.9666928052902222)]

**5 - Visualizar agrupación de vectores**

In [128]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [129]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [130]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [26]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)